### Load Dependecies and Data

In [ ]:
import pandas as pd
import torch
from torchdiffeq import odeint_adjoint as odeint
from SIR_Adjoint import fit
from matplotlib import pyplot as plt
import numpy as np
from scipy.integrate import odeint

data = pd.read_csv("/Users/omarafifi/MyFolders/Differential-Game-Theory-for-SIR-Models/Data/_normalized.csv", index_col = 0)
S, I, R = data['S'].to_numpy(), data['I'].to_numpy(),  data['R'].to_numpy()
S_0, I_0, R_0  = float(S[0]), float(I[0]), float(R[0])
S_0, I_0, R_0  
iB, ig = .01, .01
N = 1
p_0 = [iB, ig, S_0, I_0, R_0 ]

### Git the model and plot outputs

In [ ]:
X = fit(p_0, I, 3, .001)

### making a velocity plot:

In [ ]:
def forward( X, t, *args):
        # defines the SIR equations
        Β, γ  = args
        print(X)

        S, I, R = X
        N = S + I + R

        dS = -(Β/N)*S*I
        dI = (Β/N)*S*I - γ*I
        dR = γ*I

        return [dS, dI, dR]

In [ ]:
times = np.arange(0,152,1)
V = odeint(func = forward, 
           y0 = [S_0, I_0, R_0],
           t = times, 
           args = (.2,.12))

In [ ]:
plt.plot(V)
#plt.plot(V[:, 0:2])

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(4, 4))

SIRV = np.vstack ((np.linspace(0.01,.99, 20), 
                   np.linspace(0.01,.99, 20), 
                   np.zeros(20))).T


dV = np.array([forward(x_t, 0, .2,.12 ) for x_t in SIRV] )
dVS = dV[:, 0]
dVI = dV[:, 1]
x_coord, y_coord = np.meshgrid(np.linspace(0.01,.99, 20), np.linspace(0.01,.99, 20))
dVS, dVI = np.meshgrid(dVS, dVI)


fig, ax = plt.subplots()
ax.quiver(x_coord, y_coord, 
            dVS, dVI,
            angles='xy',
            scale=1.7, 
            color='r')

# Labeling the axes and the plot
plt.xlabel('Susceptible')
plt.ylabel('Infected')
plt.suptitle('Velocity Field for Susceptible and Infection Groups')
plt.axhline(y=max(V[:,1]), label = "max I(t)", c = 'black')
ax.set_title('Using Fitted Model')
plt.legend(framealpha = 1)
plt.show()